<a href="https://colab.research.google.com/github/detektor777/colab_list/blob/main/Real_ESRGAN_video.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Real-ESRGAN Video

Перейити в гугл диск и загрузить в корень диска файл с видео

https://drive.google.com/drive/ 

In [ ]:
#@title ##**Имя видео файла** { display-mode: "form" }
file_name = input("Введите имя файла и нажимете enter: ") or "input.mp4"

In [ ]:
#@title ##**Клонирование репозитория** { display-mode: "form" }
!nvidia-smi
!git clone https://github.com/xinntao/Real-ESRGAN.git
%cd Real-ESRGAN
# Set up the environment
!pip install basicsr
!pip install facexlib
!pip install gfpgan
!pip install -r requirements.txt
!python setup.py develop

In [ ]:
#@title ##**Подключение к гугл диску** { display-mode: "form" }
from google.colab import drive
drive.mount('/mnt/gdrive')


Mounted at /mnt/gdrive


In [ ]:
#@title ##**Создать папкки для кадров** { display-mode: "form" }
import os
from google.colab import files
import shutil
import os
import shutil, sys 

upload_folder = 'upload'
result_folder = 'results'

if os.path.isdir(upload_folder):
    shutil.rmtree(upload_folder)
if os.path.isdir(result_folder):
    shutil.rmtree(result_folder)
os.makedirs(upload_folder)
os.makedirs(result_folder)

In [ ]:
#@title ##**Разбить видео на кадры** { display-mode: "form" }
import cv2

path = '/mnt/gdrive/MyDrive/'

full_path = os.path.join(path, file_name)

frames_of_video = int(cv2.VideoCapture(full_path).get(cv2.CAP_PROP_FRAME_COUNT))
fps_of_video = int(cv2.VideoCapture(full_path).get(cv2.CAP_PROP_FPS))
vidcap = cv2.VideoCapture(full_path)
success,image = vidcap.read()
count = 0
success = True
while success:
  cv2.imwrite("upload/frame%09d.jpg" % count, image)
  success,image = vidcap.read()
  count += 1

In [ ]:
#@title ##**Конфигурация** { display-mode: "form" }
model_name = "RealESRGAN_x2plus" #@param ["RealESRGAN_x2plus","RealESRGAN_x4plus","RealESRNet_x4plus","realesr-general-x4v3","RealESRGAN_x4plus_anime_6B","realesr-animevideov3"]
scale = "2" #@param ["2","3","4"]
face_enhance = "0" #@param [0,1]

command = "python inference_realesrgan.py -n {} -i upload --outscale {}".format(model_name, scale)
if face_enhance:
    command += " --face_enhance"

os.system(command)

In [ ]:
#@title ##**Собрать видео из кадров** { display-mode: "form" }
import cv2
import os

output_file = "/mnt/gdrive/MyDrive/output.mp4"

# Получаем информацию о видео
cap = cv2.VideoCapture(full_path)
fps_of_video = int(cap.get(cv2.CAP_PROP_FPS))
cap.release()

# Получаем список имен файлов в директории
img_files = os.listdir(result_folder)
# Сортируем файлы в алфавитном порядке
img_files.sort()

# Получаем размеры первого кадра для создания видео
frame = cv2.imread(os.path.join(result_folder, img_files[0]))
height, width, _ = frame.shape

# Создаем объект writer, который будет писать каждый кадр в выходной видеофайл
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
writer = cv2.VideoWriter(output_file, fourcc, fps_of_video, (width, height))

# Проходим по каждому файлу
for img_file in img_files:
    # Считываем изображение из файла
    img_path = os.path.join(result_folder, img_file)
    frame = cv2.imread(img_path)

    # Записываем кадр в видеофайл
    writer.write(frame)

from moviepy.editor import *


# Освобождаем ресурсы и закрываем видеофайл
writer.release()
print('Сборка завершена')

In [ ]:
#@title ##**Добавить звук** { display-mode: "form" }
# Загружаем видео без звука
video = VideoFileClip(output_file)

# Загружаем звуковую дорожку
audio = AudioFileClip(full_path)

# Добавляем звук к видео
video = video.set_audio(audio)

# Сохраняем полученное видео
video.write_videofile('/mnt/gdrive/MyDrive/output_with_audio.mp4')

In [ ]:
#@title ##**Скачать файл** { display-mode: "form" }
from google.colab import files
files.download('/mnt/gdrive/MyDrive/output_with_audio.mp4', codec='libx264', fps=video_clip.fps, preset='medium')